In [8]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle 
import os
import sys

# from torch_geometric.data import Data, DataLoader
# from torch_geometric.nn import GCNConv, GATConv, SAGEConv, global_mean_pool
# from torch_geometric.utils import to_networkx


### DATASET 1: NEED ANNOTATIONS TO WORK (21.6GB), SKIP FOR NOW
ROOT = str(os.getcwd()).replace('/nlp', '/data/datasets/')
DATA = ROOT + os.listdir(ROOT)[0]
db_idx = {}

for i in os.listdir(DATA):
    db_idx[i] = []
    for j, x in enumerate(os.listdir(DATA + '/' + i)):
        db_idx[i].append(DATA + '/' + i + '/' + x)

all_db = list(db_idx.keys())

eg_db = pd.read_csv(db_idx[all_db[0]][0], sep='\t', engine='python', error_bad_lines=False)
eg_db.head()

label_keys = {'O': None, 'B': 'Beginning', 'I': 'Intermediate'}

Skipping line 47788: unexpected end of data


In [3]:
##NCBI Dataset --> Medical Subject Headings OR Online Mendelian Inheritance in Man
### https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/
from tabnanny import verbose
from datasets import load_dataset
dataset = load_dataset('ncbi_disease')
df_ncbi = pd.DataFrame(dataset['train'])

df_ncbi.head()

Reusing dataset ncbi_disease (/Users/devpatelio/.cache/huggingface/datasets/ncbi_disease/ncbi_disease/1.0.0/92314c7992b0b8a5ea2ad101be33f365b684a2cc011e0ffa29c691e6d32b2d03)
100%|██████████| 3/3 [00:00<00:00, 106.02it/s]


,id,tokens,ner_tags
0,0,"[Identification, of, APC2, ,, a, homologue, of...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0]"
1,1,"[The, adenomatous, polyposis, coli, (, APC, ),...","[0, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,"[Complex, formation, induces, the, rapid, degr...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,3,"[In, colon, carcinoma, cells, ,, loss, of, APC...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[Here, ,, we, report, the, identification, and...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [5]:
from transformers import BertTokenizerFast

check_type = df_ncbi.iloc[0]['ner_tags'].index(2)
print(df_ncbi.iloc[0]['tokens'][check_type]) #Check Keys

max_length = max([len(x) for x in df_ncbi['tokens']])

eg_sample = df_ncbi.iloc[0]['tokens']
label_keys = {0: None, 1: 'mesh', 2: 'omim'}

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
text_tokenized = tokenizer(eg_sample, padding='max_length', max_length=max_length, return_tensors='pt')

polyposis


Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 11.5kB/s]
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 1.56MB/s]
Downloading: 100%|██████████| 436k/436k [00:00<00:00, 3.08MB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 216kB/s]


In [6]:
print(text_tokenized)

{'input_ids': tensor([[  101,   146, 11951,  ...,     0,     0,     0],
        [  101,  1104,   102,  ...,     0,     0,     0],
        [  101, 10997,  1658,  ...,     0,     0,     0],
        ...,
        [  101,   189, 27226,  ...,     0,     0,     0],
        [  101, 17203,  1766,  ...,     0,     0,     0],
        [  101,   119,   102,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
